<a href="https://colab.research.google.com/github/LeonSilva15/fcc-machine-learning/blob/main/predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries

In [ ]:
!pip install -q git+https://github.com/tensorflow/docs

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

## Import data

In [ ]:
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv

In [ ]:
df = pd.read_csv('insurance.csv')
df.tail()

## Check missing values

In [ ]:
df.isnull().sum()

## Transform data to one-hot encoding

In [ ]:
df.dtypes

In [ ]:
# Region
df = df.join(pd.get_dummies(df.region, prefix='region')).drop('region', axis=1)
df.head()

In [ ]:
# Sex
values = df.sex.astype('category')
df['sex'] = values.cat.codes

df.head()

In [ ]:
# Smoker
values = df.smoker.astype('category')
df['smoker'] = values.cat.codes

df.head()

## Analyze the data

In [ ]:
sns.heatmap(df.corr(), annot=True, fmt='.2f')

In [ ]:
df.drop(
    ['region_northeast', 'region_northwest', 'region_southeast', 'region_southwest'],
    axis=1,
    inplace=True
)
df.drop(
    ['sex', 'children'],
    axis=1,
    inplace=True
)

In [ ]:
sns.pairplot(df)

## Prepare data

In [ ]:
size = int(len(df) * .2)
train_df = df[:-size]
test_df = df[-size:]

print(len(df), len(train_df), len(test_df))

In [ ]:
train_labels = train_df['expenses']
train_df = train_df.drop('expenses', axis=1)

test_labels = test_df['expenses']
test_df = test_df.drop('expenses', axis=1)

## Build model

In [ ]:
model = Sequential([
    layers.Input(shape=(len(train_df.keys()),)),
    layers.BatchNormalization(),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

model.compile(
    optimizer=RMSprop(0.05),
    loss='mse',
    metrics=['mae', 'mse']
)

model.summary()

In [ ]:
r = model.fit(train_df, train_labels, epochs=500,
              verbose=0, callbacks=[tfdocs.modeling.EpochDots()])

In [ ]:
res = model.evaluate(test_df, test_labels, verbose=2)
print(res)

## Testing

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_df, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_df).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
